In [35]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

from subprocess import check_output
#print(check_output(["ls", "../input"]).decode("utf8"))

# Any results you write to the current directory are saved as output.

# Load in the Client Name data
# Make sure all names uppercase (there are some mixed instances)
pd.set_option('display.max_rows', 30)
path = '/home/zongyi/bimbo_data/'
# path = '/Users/zonemercy/jupyter_notebook/bimbo_data/'
vf = pd.read_csv(path+'cliente_tabla.csv',header=0)
vf['NombreCliente'] = vf['NombreCliente'].str.upper()

In [3]:
vf['NombreCliente'].value_counts()[0:10]

NO IDENTIFICADO    281670
LUPITA               4863
MARY                 3016
LA PASADITA          2426
LA VENTANITA         2267
LA GUADALUPANA       1299
ROSY                 1246
ALEX                 1242
GABY                 1238
LA ESCONDIDA         1216
Name: NombreCliente, dtype: int64

In [4]:
# Let's also generate a list of individual word frequency across all names
def tfidf_score_list(vf2, list_len):
    from sklearn.feature_extraction.text import TfidfVectorizer
    v = TfidfVectorizer()

    vf2['New'] = 'na'
    a = " ".join(vf2['NombreCliente'])
    vf2['New'][0] = a

    tfidf = v.fit_transform(vf2['New'])

    feature_names = v.get_feature_names()

    freq = []
    doc = 0
    feature_index = tfidf[doc,:].nonzero()[1]
    tfidf_scores = zip(feature_index, [tfidf[doc, x] for x in feature_index])
    for w, s in [(feature_names[i], s) for (i, s) in tfidf_scores]:
            freq.append((w.encode('utf-8'),s))
    
    del vf2['New']
    
    import numpy as np
    names = ['word','score']
    formats = ['S50','f8']
    dtype = dict(names = names, formats=formats)
    array = np.array(freq, dtype=dtype)

    b = np.sort(array, order='score')
    
    if list_len > len(b)+1:
        list_len = len(b)+1
    for i in range(1,list_len):
        print(b[-i])

In [5]:
tfidf_score_list(vf, 200)

/usr/local/lib/python2.7/dist-packages/ipykernel/__main__.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


('no', 0.6888126004393861)
('identificado', 0.6849292193081505)
('la', 0.14990532034895288)
('el', 0.08328478631485127)
('abarrotes', 0.0800093233318993)
('de', 0.060769929775020375)
('maria', 0.046819424108208656)
('miscelanea', 0.038050812737366206)
('super', 0.03520332413080589)
('los', 0.02841652592336794)
('san', 0.025741686070920162)
('oxxo', 0.02257321668206611)
('del', 0.021668634477420134)
('garcia', 0.021376833766244014)
('hernandez', 0.02099506116912192)
('jose', 0.019835153342196838)
('lupita', 0.019261278610217134)
('gonzalez', 0.01775850494766011)
('martinez', 0.01764664800837593)
('lopez', 0.017150586799376526)
('mini', 0.015331695699712036)
('rodriguez', 0.014891562960354718)
('las', 0.01388971385198337)
('don', 0.012542567235386944)
('comodin', 0.01238450851683321)
('guadalupe', 0.012357760118308733)
('mary', 0.012270219904955897)
('jesus', 0.012024620973049328)
('ramirez', 0.011995440901931718)
('juan', 0.011966260830814104)
('casa', 0.011929785741917089)
('ag', 0.011

In [7]:
# print(vf[vf['NombreCliente'].str.contains('.*CAFE.*')])
print len(vf[vf['NombreCliente'].str.contains('NEZ|JOSE|NZO|TES')])

57514


In [9]:
# --- Begin Filtering for specific terms

# Note that the order of filtering is significant.
# For example: 
# The regex of .*ERIA.* will assign "FRUITERIA" to 'Eatery' rather than 'Fresh Market'.
# In other words, the first filters to occur have a bigger priority.

def filter_specific(vf2):
    
    # Known Large Company / Special Group Types
    vf2['NombreCliente'] = vf2['NombreCliente'].str.replace('.*REMISION.*','Consignment')
    vf2['NombreCliente'] = vf2['NombreCliente'].str.replace('.*DISTRIBUIDORA.*','Distribut')
    vf2['NombreCliente'] = vf2['NombreCliente'].replace(['.*WAL MART.*','.*SAMS CLUB.*'],'Walmart', regex=True)
    vf2['NombreCliente'] = vf2['NombreCliente'].str.replace('.*OXXO.*','Oxxo Store')
    vf2['NombreCliente'] = vf2['NombreCliente'].str.replace('.*CONASUPO.*','Govt Store')
    vf2['NombreCliente'] = vf2['NombreCliente'].str.replace('.*BIMBO.*','Bimbo Store')

    

    # General term search for a random assortment of words I picked from looking at
    # their frequency of appearance in the data and common spanish words for these categories
#     vf2['NombreCliente'] = vf2['NombreCliente'].replace(['.*NEZ.*','.*JOSE.*','.*NZO.*','.*TES.*'],'Xicans', regex=True)

    vf2['NombreCliente'] = vf2['NombreCliente'].replace(['.*COLEG.*','.*UNIV.*','.*ESCU.*','.*INSTI.*',\
                                                        '.*PREPAR.*'],'School', regex=True)
    vf2['NombreCliente'] = vf2['NombreCliente'].str.replace('.*PUESTO.*','Post')
    vf2['NombreCliente'] = vf2['NombreCliente'].replace(['.*FARMA.*','.*HOSPITAL.*','.*CLINI.*'],'Hospital', regex=True)
    vf2['NombreCliente'] = vf2['NombreCliente'].replace(['.*CAFE.*','.*CREMERIA.*','.*DULCERIA.*',\
                                                        '.*REST.*','.*BURGER.*','.*TACO.*', '.*TORTA.*',\
                                                        '.*TAQUER.*','.*HOT DOG.*',\
                                                        '.*COMEDOR.*', '.*ERIA.*','.*BURGU.*'],'Eatery', regex=True)
    vf2['NombreCliente'] = vf2['NombreCliente'].str.replace('.*SUPER.*','Supermarket')
    vf2['NombreCliente'] = vf2['NombreCliente'].replace(['.*COMERCIAL.*','.*BODEGA.*','.*DEPOSITO.*',\
                                                            '.*ABARROTES.*','.*MERCADO.*','.*CAMBIO.*',\
                                                        '.*MARKET.*','.*MART .*','.*MINI .*',\
                                                        '.*PLAZA.*','.*MISC.*','.*ELEVEN.*','.*SEVEN.*','.*EXP.*',\
                                                         '.*SNACK.*', '.*PAPELERIA.*', '.*CARNICERIA.*',\
                                                         '.*LOCAL.*','.*COMODIN.*','.*PROVIDENCIA.*'
                                                        ],'General Market'\
                                                       , regex=True)

    vf2['NombreCliente'] = vf2['NombreCliente'].replace(['.*VERDU.*','.*FRUT.*'],'Fresh Market', regex=True)
    vf2['NombreCliente'] = vf2['NombreCliente'].replace(['.*HOTEL.*','.*MOTEL.*'],'Hotel', regex=True)
    vf2['NombreCliente'] = vf2['NombreCliente'].replace(['.*NEZ.*','.*JOSE.*','.*NZO.*','.*TES.*'],'Xicans', regex=True)
    vf2['NombreCliente'] = vf2['NombreCliente'].str.replace('.*MODELOR.*','Modelor')
    vf2['NombreCliente'] = vf2['NombreCliente'].str.replace('.*ARTELI.*','Arteli')
    vf2['NombreCliente'] = vf2['NombreCliente'].str.replace('.*CALIMAX.*','Calimax')

In [10]:
filter_specific(vf)

In [12]:
# --- Begin filtering for more general terms
# The idea here is to look for names with particles of speech that would
# not appear in a person's name.
# i.e. "Individuals" should not contain any participles or numbers in their names.
def filter_participle(vf2):
    vf2['NombreCliente'] = vf2['NombreCliente'].replace([
            '.*LA .*','.*EL .*','.*DE .*','.*LOS .*','.*DEL .*','.*Y .*', '.*SAN .*', '.*SANTA .*',\
            '.*AG .*','.*LAS .*','.*MI .*','.*MA .*', '.*II.*', '.*[0-9]+.*'\
    ],'Small Franchise', regex=True)

In [13]:
filter_participle(vf)

In [15]:
# Any remaining entries should be "Individual" Named Clients, there are some outliers.
# More specific filters could be used in order to reduce the percentage of outliers in this final set.
def filter_remaining(vf2):
    def function_word(data):
        # Avoid the single-words created so far by checking for upper-case
        if (data.isupper()) and (data != "NO IDENTIFICADO"): 
            return 'Individual'
        else:
            return data
    vf2['NombreCliente'] = vf2['NombreCliente'].map(function_word)

In [16]:
filter_remaining(vf)

In [18]:
vf['NombreCliente'].value_counts()

Individual         335692
NO IDENTIFICADO    281670
Small Franchise    154007
General Market      66378
Eatery              30412
Xicans              22352
Supermarket         16015
Oxxo Store           9313
Hospital             5782
School               5705
Post                 2667
Modelor              1280
Hotel                1127
Fresh Market         1068
Govt Store            959
Bimbo Store           320
Distribut             288
Walmart               220
Calimax                73
Arteli                 20
Consignment            14
Name: NombreCliente, dtype: int64

In [20]:
from sklearn import preprocessing
le = preprocessing.LabelEncoder()
le.fit(vf['NombreCliente'])

print le.classes_

vf['c_clt'] = le.transform(vf['NombreCliente']) 


['Arteli' 'Bimbo Store' 'Calimax' 'Consignment' 'Distribut' 'Eatery'
 'Fresh Market' 'General Market' 'Govt Store' 'Hospital' 'Hotel'
 'Individual' 'Modelor' 'NO IDENTIFICADO' 'Oxxo Store' 'Post' 'School'
 'Small Franchise' 'Supermarket' 'Walmart' 'Xicans']


In [22]:
vf.head()

,Cliente_ID,NombreCliente,c_clt
0,0,Individual,11
1,1,Oxxo Store,14
2,2,Individual,11
3,3,Small Franchise,17
4,4,Small Franchise,17


In [27]:
len(vf) #935362

935362

In [24]:
vf = vf[['Cliente_ID','c_clt']]

In [32]:
# vf = vf[vf['Cliente_ID'].unique()]
vf = vf.drop_duplicates(subset='Cliente_ID', keep='last')

In [33]:
vf

,Cliente_ID,c_clt
0,0,11
1,1,14
2,2,11
3,3,17
5,4,17
6,5,17
7,6,11
8,7,17
9,8,18
10,9,18


In [34]:
vf.to_csv(path+'clients.csv',index=False)